In [1]:
import itertools
import re
import pandas as pd

# Defining a function to get w-2, w-1, w+1, w+2. 

No formatting done, special characters are considered seperate words.

In [2]:
def clean_data(df):
    
    #Changing values stored as <head>value<\head> to value.
    
    df['w-2'] = df['w-2'].str.replace('<head>', '')
    df['w-2'] = df['w-2'].str.replace('</head>', '')

    df['w-1'] = df['w-1'].str.replace('<head>', '')
    df['w-1'] = df['w-1'].str.replace('</head>', '')

    df['w+1'] = df['w+1'].str.replace('<head>', '')
    df['w+1'] = df['w+1'].str.replace('</head>', '')

    df['w+2'] = df['w+2'].str.replace('<head>', '')
    df['w+2'] = df['w+2'].str.replace('</head>', '')
    
    return df

In [3]:
def get_matching_string(infile, outfile):
    
    #Reading the file instance by instance
    
    with open(infile, 'r') as f:
        text = f.read()
        text = text.split("<instance id=")
    
    fout = open(outfile, 'w')

    featurelist = ''
    
    for i in text:
        
        #Matching the first occurence as we are only considering one word per instance
    
        matches = re.search('(\S+\s){0,2}(<head>\w+</head>)+(\s\S+){0,2}', i)
    
        result = matches.group(0) if matches else ""
    
        featurelist = featurelist + result + '\n'
    
    
    fout.write(featurelist)
    
    fout.close()
    
    #Cleaning using pandas dataframe.
    
    df = pd.read_csv(outfile, names=['w-2','w-1','w','w+1', 'w+2'], delimiter = ' ')
    df = df.drop('w', 1)
    
    df = clean_data(df)
    
    return df

In [4]:
def get_key(keyfile, key):
    
    #Getting the keys from the file.
    df = pd.read_csv(keyfile, names=['name', 'instance','senseid', 'senseid2'], delimiter = ' ')
    
    #Considering only the first senseid.
    
    df = df.drop('senseid2', 1)
    
    #Only considering the particluar key.
    
    df = df.loc[df['name'] == key]

    return df

# Defining a function to extract all the examples of the given words. 

Code to extract infromation between two word instances was taken from: "https://stackoverflow.com/questions/18865058/extract-values-between-two-strings-in-a-text-file-using-python"

In [5]:
def extract_content(infile, outfile, start, end, featurefile, keyfile, key):
    
    #Dividing our data accroding to the given name. Creating seperate files for each name
    with open(infile, 'r') as f, open(outfile, 'w') as fout:
        while True:
            it = itertools.dropwhile(lambda line: line.strip() != start, f)
            if next(it, None) is None: break
            fout.writelines(itertools.takewhile(lambda line: line.strip() != end, it))
            
    #Getting the features and the keys. 
    
    df1 = get_matching_string(outfile, featurefile)
    df2 = get_key(keyfile, key)
    
    #Joining two dataframes.
    
    df1[['name', 'instance', 'senseid']]= df2[['name', 'instance', 'senseid']].values
    
    return df1

# Defining all the names

In [6]:
infile1 = 'EnglishLS.train/EnglishLS.train'
infile2 = 'EnglishLS.test/EnglishLS.test'


outfile1 = 'arms_train.txt'
outfile2 = 'difficulty_train.txt'
outfile3 = 'interest_train.txt'
outfile4 = 'arms_test.txt'
outfile5 = 'difficulty_test.txt'
outfile6 = 'interest_test.txt'


start1 = '<lexelt item="arm.n">'
start2 = '<lexelt item="difficulty.n">'
start3 = '<lexelt item="interest.n">'


end = '</lexelt>'


featurefile1 = 'arms_train_features.txt'
featurefile2 = 'difficulty_train_features.txt'
featurefile3 = 'interest_train_features.txt'

featurefile4 = 'arms_test_features.txt'
featurefile5 = 'difficulty_test_features.txt'
featurefile6 = 'interest_test_features.txt'

keyfile1 = 'EnglishLS.train/EnglishLS.train.key'
keyfile2 = 'EnglishLS.test/EnglishLS.test.key'

key1 = 'arm.n'
key2 = 'difficulty.n'
key3 = 'interest.n'

# Calling the functions on all 3 training and testing data.

In [7]:
df1 = extract_content(infile1, outfile1, start1, end, featurefile1, keyfile1, key1)
df2 = extract_content(infile1, outfile2, start2, end, featurefile2, keyfile1, key2)
df3 = extract_content(infile1, outfile3, start3, end, featurefile3, keyfile1, key3)

In [8]:
df4 = extract_content(infile2, outfile4, start1, end, featurefile4, keyfile2, key1)
df5 = extract_content(infile2, outfile5, start2, end, featurefile5, keyfile2, key2)
df6 = extract_content(infile2, outfile6, start3, end, featurefile6, keyfile2, key3)

In [9]:
df1

,w-2,w-1,w+1,w+2,name,instance,senseid
0,with,their,chained,to,arm.n,arm.n.bnc.00001857,arm%1:08:00::
1,under,his,and,come,arm.n,arm.n.bnc.00003355,arm%1:08:00::
2,raised,his,involuntarily,",",arm.n,arm.n.bnc.00004403,arm%1:08:00::
3,under,your,and,wishing,arm.n,arm.n.bnc.00005894,arm%1:08:00::
4,searching,for,.,This,arm.n,arm.n.bnc.00008679,arm%1:06:01::
...,...,...,...,...,...,...,...
261,with,one,up,a,arm.n,arm.n.bnc.00122694,arm%1:08:00::
262,losing,one,or,one,arm.n,arm.n.bnc.00122695,arm%1:08:00::
263,lost,an,or,leg,arm.n,arm.n.bnc.00125007,arm%1:08:00::
264,",",his,by,his,arm.n,arm.n.bnc.00125065,arm%1:08:00::


In [10]:
df2

,w-2,w-1,w+1,w+2,name,instance,senseid
0,of,the,is,that,difficulty.n,difficulty.n.bnc.00000617,difficulty%1:26:00::
1,I,have,in,continuing,difficulty.n,difficulty.n.bnc.00000729,difficulty%1:04:00::
2,can,produce,in,the,difficulty.n,difficulty.n.bnc.00001154,difficulty%1:09:02::
3,although,the,include,the,difficulty.n,difficulty.n.bnc.00002701,difficulty%1:26:00::
4,in,serious,about,knowing,difficulty.n,difficulty.n.bnc.00002734,difficulty%1:09:02::
5,may,have,in,deciding,difficulty.n,difficulty.n.bnc.00002943,difficulty%1:04:00::
6,most,obvious,is,that,difficulty.n,difficulty.n.bnc.00003167,difficulty%1:07:00::
7,performance,pose,for,the,difficulty.n,difficulty.n.bnc.00003238,difficulty%1:09:02::
8,",",the,are,complicated,difficulty.n,difficulty.n.bnc.00003400,difficulty%1:09:02::
9,note,about,of,attribution,difficulty.n,difficulty.n.bnc.00003417,difficulty%1:09:02::


In [11]:
df3

,w-2,w-1,w+1,w+2,name,instance,senseid
0,Serving,the,of,Haemophiliacs,interest.n,interest.n.bnc.00000034,interest%1:07:01::
1,and,the,of,clients,interest.n,interest.n.bnc.00000347,interest%1:07:01::
2,and,shared,",",not,interest.n,interest.n.bnc.00000571,interest%1:04:01::
3,building,society,on,which,interest.n,interest.n.bnc.00000714,interest%1:21:00::
4,building,society,on,which,interest.n,interest.n.bnc.00000823,interest%1:21:00::
...,...,...,...,...,...,...,...
180,shown,increasing,in,it,interest.n,interest.n.bnc.00088440,interest%1:09:00::
181,one,'s,to,have,interest.n,interest.n.bnc.00088848,interest%1:07:01::
182,the,public,.,The,interest.n,interest.n.bnc.00090079,interest%1:07:01::
183,only,of,to,three,interest.n,interest.n.bnc.00090901,interest%1:07:02::


In [12]:
df4

,w-2,w-1,w+1,w+2,name,instance,senseid
0,with,six,as,Siva,arm.n,arm.n.bnc.00003787,arm%1:08:00::
1,all,his,",",And,arm.n,arm.n.bnc.00005920,arm%1:08:00::
2,of,folded,",",The,arm.n,arm.n.bnc.00006174,arm%1:08:00::
3,above,the,of,the,arm.n,arm.n.bnc.00012302,arm%1:06:03::
4,the,sparging,at,the,arm.n,arm.n.bnc.00012816,U
...,...,...,...,...,...,...,...
128,side,-,",",sometimes,arm.n,arm.n.bnc.00122491,arm%1:06:01::
129,waving,his,about,and,arm.n,arm.n.bnc.00123841,arm%1:08:00::
130,drawing,his,in,a,arm.n,arm.n.bnc.00124279,arm%1:08:00::
131,washed,my,in,the,arm.n,arm.n.bnc.00125195,arm%1:08:00::


In [13]:
df5

,w-2,w-1,w+1,w+2,name,instance,senseid
0,produce,great,in,the,difficulty.n,difficulty.n.bnc.00000264,difficulty%1:04:00::
1,encounter,many,with,our,difficulty.n,difficulty.n.bnc.00002181,difficulty%1:09:02::
2,of,her,is,to,difficulty.n,difficulty.n.bnc.00002364,difficulty%1:07:00::
3,find,more,in,following,difficulty.n,difficulty.n.bnc.00002727,difficulty%1:04:00::
4,seldom,without,),when,difficulty.n,difficulty.n.bnc.00002814,difficulty%1:26:00::
5,:,his,is,assessing,difficulty.n,difficulty.n.bnc.00003064,difficulty%1:09:02::
6,The,first,is,",",difficulty.n,difficulty.n.bnc.00003419,difficulty%1:09:02::
7,One,difficulty,describing,NaN,difficulty.n,difficulty.n.bnc.00003456,difficulty%1:09:02::
8,constantly,presented,to,critics,difficulty.n,difficulty.n.bnc.00003685,difficulty%1:09:02::
9,some,tormenting,",",His,difficulty.n,difficulty.n.bnc.00004690,difficulty%1:26:00::


In [14]:
df6

,w-2,w-1,w+1,w+2,name,instance,senseid
0,",",mortgage,relief,",",interest.n,interest.n.bnc.00000703,interest%1:21:00::
1,",",economic,and,religious,interest.n,interest.n.bnc.00001296,interest%1:07:01::
2,Her,interest,been,NaN,interest.n,interest.n.bnc.00002352,interest%1:04:01::
3,and,persistent,which,human,interest.n,interest.n.bnc.00002626,interest%1:09:00::
4,indicated,an,in,the,interest.n,interest.n.bnc.00002778,interest%1:09:00::
...,...,...,...,...,...,...,...
88,the,public,for,challenges,interest.n,interest.n.bnc.00090159,interest%1:07:01::
89,serve,the,of,the,interest.n,interest.n.bnc.00090211,interest%1:07:01::
90,a,financial,in,United,interest.n,interest.n.bnc.00090467,interest%1:21:03::
91,a,financial,in,another,interest.n,interest.n.bnc.00090469,interest%1:21:03::


Still needs to be done:

What to replace Nan values with.
What to do when 's is considered, and not the word.

### Discarded Code

In [15]:
# def get_matching_string(infile, outfile):
    
#     with open(infile, 'r') as f:
#         text = f.read()
    
#         matches = re.finditer('(\S+\s){0,2}(<head>\w+</head>)+(\s\S+){0,2}', text)
        
#     with open(outfile, 'w') as fout:
#         for m in matches:
#             fout.write(m.group(0) + '\n')
        
#     df = pd.read_csv(outfile, names=['w-2','w-1','w','w+1', 'w+2'], delimiter = ' ')
#     df = df.drop('w', 1)
    
#     return df

In [16]:
# def get_key(infile, outfile):
    
#     with open(infile, 'r') as f:
#         text = f.read()

#         matches = re.finditer('instance="\S+"\ssenseid="\S+"', text)
        
#     with open(outfile, 'w') as fout:
#         for m in matches:
#             fout.write(m.group(0) + '\n')
    

#     df = pd.read_csv(outfile, names=['instance','senseid'], delimiter = ' ')
    
#     df[['insta','instakey']] = df['instance'].str.split('=', 1, expand = True)
#     df[['sense','sensekey']] = df['senseid'].str.split('=', 1, expand = True)
    
#     df = df.drop(['instance','senseid','insta','sense'], 1)
    
#     df['instakey'] = df['instakey'].str.strip('"')
#     df['sensekey'] = df['sensekey'].str.strip('"')

#     df.drop_duplicates(subset=['instakey'])
    
#     return df

In [17]:
# def get_key(infile, outfile):
    
#     with open(infile, 'r') as f:
#         text = f.read()
#         text = text.split("<instance id=")
    
#     fout = open(outfile, 'w')
    
#     keylist = ''
    
#     for i in text:
        
#         matches = re.search('instance="\S+"\ssenseid="\S+"', i)
    
#         result = matches.group(0) if matches else ""
    
#         keylist = keylist + result + '\n'
    
#     fout.write(keylist)
    
#     fout.close()
    
#     #Cleaning using pandas dataframe.
    
#     df = pd.read_csv(outfile, names=['instance','senseid'], delimiter = ' ')
    
#     df[['insta','instakey']] = df['instance'].str.split('=', 1, expand = True)
#     df[['sense','sensekey']] = df['senseid'].str.split('=', 1, expand = True)
    
#     df = df.drop(['instance','senseid','insta','sense'], 1)
    
#     df['instakey'] = df['instakey'].str.strip('"')
#     df['sensekey'] = df['sensekey'].str.strip('"')

#     df.drop_duplicates(subset=['instakey'])
    
#     return df
    
    

In [18]:
# import re
# text = "the women marathon <head>unknowns</head> introduced at the summer olympics los angeles usa and <head>unknown</head> won"
# matches = re.finditer('(\S+\s){0,2}(<head>\w+</head>)+(\s\S+){0,2}', text)

# for m in matches:
#     print(m.group())

In [19]:
# from bs4 import BeautifulSoup

# with open('EnglishLS.train/EnglishLS.dictionary.xml', 'r') as f:

#     contents = f.read()

#     soup = BeautifulSoup(contents, 'lexelt')

#     #print(soup.find('ul', attrs={ 'id' : 'mylist'}))
#     print(soup.find('instance', id='arm.n'))

In [20]:
# def get_dataframe(file):
    
#     df = pd.read_csv(file, names=['w-2','w-1','w','w+1', 'w+2'], delimiter = ' ')
#     df = df.drop('w', 1)
    
#     return df